# Change Detection (CD) with Random Forest (RF)

### Post-Classification CD using RF classifier
### Using Google Earth Engine Python API and NICFI Normalized Analytic Basemap from December 2015, December 2017, December 2019, and December 2021

Author: Finn Geiger\
Date: April 6th 2023\
Contact:
- https://github.com/finn-geiger
- https://www.linkedin.com/in/finn-geiger-b1329a20b/

### Steps:
1. Importing the datasets and classification result of 2015
2. OPIT classification for 2017, 2019, and 2021
3. CD for TOI 2015 to 2017
4. CD for TOI 2017 to 2019
5. CD for TOI 2019 to 2021
6. Final products

### 1 Import and setup
#### 1.1 Importing the required libraries and packages

In [15]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee
import os
import time
import pandas as pd
from tabulate import tabulate
#%pip install tabulate


The following classes and landcover IDs will be used:

In [16]:
info = {'Class name': ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies'],
        'landcover ID': [1, 2, 3, 4, 5, 6, 7]}

print(tabulate(info, headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════╕
│ Class name   │   landcover ID │
╞══════════════╪════════════════╡
│ Informal     │              1 │
├──────────────┼────────────────┤
│ Formal       │              2 │
├──────────────┼────────────────┤
│ Industrial   │              3 │
├──────────────┼────────────────┤
│ Roads        │              4 │
├──────────────┼────────────────┤
│ Vacant land  │              5 │
├──────────────┼────────────────┤
│ Vegetation   │              6 │
├──────────────┼────────────────┤
│ Water-bodies │              7 │
╘══════════════╧════════════════╛


##### When first using the GEE Python API the user must authenticate and initialize the environment by using the following two lines of codes:

In [17]:
#ee.Authenticate() 
#ee.Initialize()

In [18]:
# creating the map
Map = geemap.Map()

# loading the interactive map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### 1.2 Importing the datasets from GEE assets and data catalog and clipping the basemap to the AOI

In [19]:
# Loading the Base scene
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')

# Filter basemaps by date and get the first image from filtered results
basemap_2017_12 = nicfi.filter(ee.Filter.date('2017-12-01','2018-01-01')).first()
basemap_2019_12 = nicfi.filter(ee.Filter.date('2019-12-01','2029-01-01')).first()
basemap_2021_12 = nicfi.filter(ee.Filter.date('2021-12-01','2022-01-01')).first()

# importing the classified scene from 2015
classified_TOI1_2015 = ee.Image('users/s85315/masterthesis/change_detection_results/classified_TOI1_RF')

# Visualizing the scene
vis_params = {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}

# Adding the basemap to the map
Map.centerObject(basemap_2017_12, 4)
# Map.addLayer(basemap_2017_12, vis_params, '2017-12 mosaic', False)
# Map.addLayer(basemap_2019_12, vis_params, '2019-12 mosaic', False)
# Map.addLayer(basemap_2021_12, vis_params, '2021-12 mosaic', False)

In [20]:
# Loading the AOI and Masking the base scene
vis_params_aoi = {'color': 'blue'}
aoi_windhoek = ee.FeatureCollection('users/s85315/masterthesis/Study_Area_Windhoek')

# converting the FeatureCollection to Geometry for export
aoi_geom = aoi_windhoek.geometry()

# Adding the AOI to the map
Map.addLayer(aoi_windhoek, vis_params_aoi, 'AOI')
Map.centerObject(aoi_windhoek, 12)

In [21]:
# clipping the clipped_TOI1_2015 to the AOI
clipped_TOI2_2017 = basemap_2017_12.clipToCollection(aoi_windhoek)
clipped_TOI3_2019 = basemap_2019_12.clipToCollection(aoi_windhoek)
clipped_TOI4_2021 = basemap_2021_12.clipToCollection(aoi_windhoek)

Map.addLayer(clipped_TOI2_2017, vis_params, 'clipped_2017', False)
Map.addLayer(clipped_TOI3_2019, vis_params, 'clipped_2019', False)
Map.addLayer(clipped_TOI4_2021, vis_params, 'clipped_2021', False)

In [22]:
# importing training data samples and adding them to the map
# Transforming the Geometries into FeatureCollections and applying properties
training_samples_2017 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI2_2017_RPoints')
training_samples_2019 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI3_2019_RPoints')
training_samples_2021 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI4_2021_RPoints')

# adding the layers to the map
# Map.addLayer(training_samples_2017, {}, 'Training Samples for 2017')
# Map.addLayer(training_samples_2019, {}, 'Training Samples for 2019')
# Map.addLayer(training_samples_2021, {}, 'Training Samples for 2021')

### 2 OPIT Classification with RF for TOIs 2017, 2019, and 2021

#### 2.1 Configuration and creation of the empty RF classifier

In [23]:
# creating variable for parameter settings
# Mtry will be set to default
RF_param = 550

classifier_params = {
              'numberOfTrees':RF_param, # 	Ntree; The number of decision trees to create.
              'variablesPerSplit':None, # Mtry; The number of variables per split. If unspecified, uses the square root of the number of variables.
              'minLeafPopulation':1, # smallest sample size possible per leaf
              'bagFraction':0.5, #The fraction of input to bag per tree.
              'maxNodes':None, # the number of individual decision tree models
              'seed': 0}  # The randomization seed.

#### 2.2 Applying training samples on the scenes

##### 2.2.1 TOI2 - 2017

In [24]:
# adding the training samples to the basescene
training_2017 = clipped_TOI2_2017.sampleRegions(**{
    'collection': training_samples_2017, 
    'properties': ['landcover'], 
    'scale': 4.77,
    'projection': 'EPSG:32733'
})

# creating the classifier using RF
classifier_2017 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2017,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI2_2017.bandNames()
})

##### 2.2.2 TOI3 - 2019

In [51]:
# adding the training samples to the basescene
training_2019 = clipped_TOI3_2019.sampleRegions(**{
    'collection': training_samples_2019, 
    'properties': ['landcover'], 
    'projection': 'EPSG:32733',
    'scale': 4.77
})

# creating the classifier using RF
classifier_2019 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2019,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI3_2019.bandNames()
})

##### 2.2.3 TOI4 - 2021

In [52]:
# adding the training samples to the basescene
training_2021 = clipped_TOI4_2021.sampleRegions(**{
    'collection': training_samples_2021, 
    'properties': ['landcover'], 
    'scale': 4.77,
    'projection': 'EPSG:32733'
})

# creating the classifier using RF
classifier_2021 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2021,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI4_2021.bandNames()
})

#### 2.3 classifying the basescene and visualizing the product

In [53]:
# classifying the basescene using the created classifier
classified_TOI2_2017 = clipped_TOI2_2017.classify(classifier_2017)
classified_TOI3_2019 = clipped_TOI3_2019.classify(classifier_2019)
classified_TOI4_2021 = clipped_TOI4_2021.classify(classifier_2021)

# creating the visualization parameters
palette = ['c43c39', 'e5b636', '2f2f2f', 'aaaaaa', 'b08e7a', '85b66f', 'a5bfdd']
vis_params_classified = {'min': 1, 'max': 7, 'palette': palette}


# Map.addLayer(classified_TOI1_2015, vis_params_classified, 'classified scene - 2015')
# Map.addLayer(classified_TOI2_2017, vis_params_classified, 'classified scene - 2017')
# Map.addLayer(classified_TOI3_2019, vis_params_classified, 'classified scene - 2019')
# Map.addLayer(classified_TOI4_2021, vis_params_classified, 'classified scene - 2021')


#### 2.4 Exporting the results

##### 2.4.1 Exporting to Google Drive

In [28]:
# # exporting to Google drive with GEE API
# export_TOI2 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI2_2017,
#     'description': 'classified_TOI2_RF_test', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'crs': 'EPSG:32733',
#     'scale': 4.77,
#     'region': aoi_geom
# })

# # starting the process
# export_TOI2.start()

# # tracking the process
# while export_TOI2.active():
#   print('Polling for task (id: {}).'.format(export_TOI2.id))
#   time.sleep(5)

Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id: CMV7TCS57NP5X4WNFQPA3YHT).
Polling for task (id

In [29]:
# # exporting to Google drive with GEE API
# export_TOI3 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI3_2019,
#     'description': 'classified_TOI3_RF_test', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_TOI3.start()

# # tracking the process
# while export_TOI3.active():
#   print('Polling for task (id: {}).'.format(export_TOI3.id))
#   time.sleep(5)

Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id: 4OLQRDJ7MBYZ4J2PRO7NCEI4).
Polling for task (id

In [54]:
# # exporting to Google drive with GEE API
# export_TOI4 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI4_2021,
#     'description': 'classified_TOI4_RF', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_TOI4.start()

# # tracking the process
# while export_TOI4.active():
#   print('Polling for task (id: {}).'.format(export_TOI4.id))
#   time.sleep(5)

Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id: RQNQ4ESNDKO3X7KCCPGUW35C).
Polling for task (id

##### 2.4.2 Exporting to Asset

In [31]:
# # exporting to Google Asset
# export_TOI2 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI2_2017,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI2_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI2.start()

# # tracking the process
# while export_TOI2.active():
#   print('Polling for task (id: {}).'.format(export_TOI2.id))
#   time.sleep(5)

In [32]:
# # exporting to Google Asset
# export_TOI3 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI3_2019,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI3_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI3.start()

# # tracking the process
# while export_TOI3.active():
#   print('Polling for task (id: {}).'.format(export_TOI3.id))
#   time.sleep(5)

In [33]:
# # exporting to Google Asset
# export_TOI4 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI4_2021,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI4_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI4.start()

# # tracking the process
# while export_TOI4.active():
#   print('Polling for task (id: {}).'.format(export_TOI4.id))
#   time.sleep(5)

### 3 CD for TOI 2015 to 2017

#### 3.1 Performing the CD analysis

In [34]:
changed_areas_2015_2017 = classified_TOI2_2017.subtract(classified_TOI1_2015).neq(0)
changed_areas_2017_2019 = classified_TOI3_2019.subtract(classified_TOI2_2017).neq(0)
changed_areas_2019_2021 = classified_TOI4_2021.subtract(classified_TOI3_2019).neq(0)
changed_areas_2015_2021 = classified_TOI4_2021.subtract(classified_TOI1_2015).neq(0)

# changed_areas_2015_2017.getInfo()

In [35]:
vis_params_changed = {'min':0, 'max':1, 'palette': ['white', 'red']}


Map.addLayer(changed_areas_2015_2017, vis_params_changed, 'Changed areas 2015 to 2017')
Map.addLayer(changed_areas_2017_2019, vis_params_changed, 'Changed areas 2017 to 2019')
Map.addLayer(changed_areas_2019_2021, vis_params_changed, 'Changed areas 2019 to 2021')
Map.addLayer(changed_areas_2015_2021, vis_params_changed, 'Changed areas 2015 to 2021')

transition_2015_2017 = classified_TOI1_2015.multiply(100).add(classified_TOI2_2017).rename('transitions')
transition_2017_2019 = classified_TOI2_2017.multiply(100).add(classified_TOI3_2019).rename('transitions')
transition_2019_2021 = classified_TOI3_2019.multiply(100).add(classified_TOI4_2021).rename('transitions')
transition_2015_2021 = classified_TOI1_2015.multiply(100).add(classified_TOI4_2021).rename('transitions')



In [36]:
# zones = ee.Image(changed_areas_2015_2017).eq(1)

# test = zones.reduceToVectors(**{
#     'geometry': aoi_geom,
#     'scale': 4.77,
#     'crs': 'EPSG:32733'
# })

# Map.addLayer(zones, vis_params_changed, "Zones")
# Map.addLayer(test, vis_params_aoi, "Test")

# joined = test.union()
# Map.addLayer(joined, {'color':'red'}, name="JOIN")


#### Calculation of transitions in Pixels

#### Transition for 2015 to 2017

In [55]:
# calculating the size per 
transition_matrix_CD1 = transition_2015_2017.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10
  })

transitions_2015_2017 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2015_2017 = pd.DataFrame(transitions_2015_2017, index=[0])
df_transitions_2015_2017_long = pd.melt(df_transitions_2015_2017, var_name='From-To', value_name="Pixel")
df_transitions_2015_2017_long['Pixel'] = df_transitions_2015_2017_long['Pixel'].astype(int)


print(df_transitions_2015_2017_long.head())
df_transitions_2015_2017_long.to_csv('./data/transitions_pixels_2015_2017.csv', index=False, sep=";")


  From-To   Pixel
0     101  147445
1     102   54335
2     103   34905
3     104   70090
4     105   43585


#### Calculating the transition values per class for 2015-2017

In [38]:
area_changed_CD1_sqm = ee.Image.pixelArea().addBands(transition_2015_2017)
area_changed_CD1_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2015_2017)


# getting the values per class
areas_changed_CD1_sqm_class = area_changed_CD1_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733'
    # 'tileScale': 16,
    # 'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD1_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2015_2017.csv', sep=";", index=False)

[{'sum': 3625858.252694889, 'transitions': 101}, {'sum': 1199871.3953720767, 'transitions': 102}, {'sum': 709892.6857879788, 'transitions': 103}, {'sum': 1977379.9721203074, 'transitions': 104}, {'sum': 1270680.1807777854, 'transitions': 105}, {'sum': 106489.06981997771, 'transitions': 106}, {'sum': 204.7095489501953, 'transitions': 107}, {'sum': 2021279.1587877162, 'transitions': 201}, {'sum': 2586434.1728143357, 'transitions': 202}, {'sum': 839678.3277381074, 'transitions': 203}, {'sum': 1829355.6465271893, 'transitions': 204}, {'sum': 497836.0273680743, 'transitions': 205}, {'sum': 55122.24961551592, 'transitions': 206}, {'sum': 136.47370719909668, 'transitions': 207}, {'sum': 491941.7379461251, 'transitions': 301}, {'sum': 962255.7259124905, 'transitions': 302}, {'sum': 765548.8217421214, 'transitions': 303}, {'sum': 370325.87897659, 'transitions': 304}, {'sum': 35378.91639775295, 'transitions': 305}, {'sum': 7719.345889745974, 'transitions': 306}, {'sum': 136.47616577148438, 'tran

In [39]:
# calculating the size per 
transition_matrix_CD2 = transition_2017_2019.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10
  })

transitions_2017_2019 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2017_2019 = pd.DataFrame(transitions_2017_2019, index=[0])
df_transitions_2017_2019_long = pd.melt(df_transitions_2017_2019, var_name='From-To', value_name="Pixel")
df_transitions_2017_2019_long['Pixel'] = df_transitions_2017_2019_long['Pixel'].astype(int)


print(df_transitions_2017_2019_long.head())
df_transitions_2017_2019_long.to_csv('./data/transitions_pixels_2017_2019.csv', index=False, sep=";")


  From-To   Pixel
0     101  189647
1     102   61558
2     103   37005
3     104  103103
4     105   64244


In [40]:
area_changed_CD2_sqm = ee.Image.pixelArea().addBands(transition_2017_2019)
area_changed_CD2_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2017_2019)


# getting the values per class
areas_changed_CD2_sqm_class = area_changed_CD2_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733',
    'tileScale': 16,
    'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD2_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2017_2019.csv', sep=";", index=False)

[{'sum': 4030994.901947515, 'transitions': 101}, {'sum': 856272.8950868868, 'transitions': 102}, {'sum': 810769.0486038283, 'transitions': 103}, {'sum': 2899439.2973354487, 'transitions': 104}, {'sum': 1973279.4406526454, 'transitions': 105}, {'sum': 167360.48219146728, 'transitions': 106}, {'sum': 227.4703598022461, 'transitions': 107}, {'sum': 751387.9852610644, 'transitions': 201}, {'sum': 3182524.769529021, 'transitions': 202}, {'sum': 647825.6418154623, 'transitions': 203}, {'sum': 1811154.6130795197, 'transitions': 204}, {'sum': 686035.8258737302, 'transitions': 205}, {'sum': 100500.5552870208, 'transitions': 206}, {'sum': 409.44758224487305, 'transitions': 207}, {'sum': 615697.7563844793, 'transitions': 301}, {'sum': 738222.2096057743, 'transitions': 302}, {'sum': 895784.0366549398, 'transitions': 303}, {'sum': 316413.73357229045, 'transitions': 304}, {'sum': 11964.715456008911, 'transitions': 305}, {'sum': 1342.0601768493652, 'transitions': 306}, {'sum': 113.72953033447266, 'tr

In [41]:
# calculating the size per 
transition_matrix_CD3 = transition_2019_2021.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10
  })

transitions_2019_2021 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2019_2021 = pd.DataFrame(transitions_2019_2021, index=[0])
df_transitions_2019_2021_long = pd.melt(df_transitions_2019_2021, var_name='From-To', value_name="Pixel")
df_transitions_2019_2021_long['Pixel'] = df_transitions_2019_2021_long['Pixel'].astype(int)

print(df_transitions_2019_2021_long.head())
df_transitions_2019_2021_long.to_csv('./data/transitions_pixels_2019_2021.csv', index=False, sep=";")


  From-To   Pixel
0     101  189647
1     102   61558
2     103   37005
3     104  103103
4     105   64244


In [42]:
area_changed_CD3_sqm = ee.Image.pixelArea().addBands(transition_2019_2021)
area_changed_CD3_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2019_2021)


# getting the values per class
areas_changed_CD3_sqm_class = area_changed_CD3_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733',
    'tileScale': 16,
    'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD3_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2019_2021.csv', sep=";", index=False)

[{'sum': 3988117.0130767147, 'transitions': 101}, {'sum': 983411.4149369932, 'transitions': 102}, {'sum': 480118.61289343744, 'transitions': 103}, {'sum': 1419632.1324560652, 'transitions': 104}, {'sum': 1195697.5092193305, 'transitions': 105}, {'sum': 57664.435903137804, 'transitions': 106}, {'sum': 1522917.9825707306, 'transitions': 201}, {'sum': 2900949.9154788223, 'transitions': 202}, {'sum': 802201.6037497501, 'transitions': 203}, {'sum': 805812.371485482, 'transitions': 204}, {'sum': 167789.02837834827, 'transitions': 205}, {'sum': 4739.744430803785, 'transitions': 206}, {'sum': 980808.754667312, 'transitions': 301}, {'sum': 712814.3106835908, 'transitions': 302}, {'sum': 699276.3701369043, 'transitions': 303}, {'sum': 507987.2750725316, 'transitions': 304}, {'sum': 281737.44060843973, 'transitions': 305}, {'sum': 22973.35399055481, 'transitions': 306}, {'sum': 15216.736364364624, 'transitions': 307}, {'sum': 3284613.622054208, 'transitions': 401}, {'sum': 1146417.784442004, 'tra

In [43]:
# calculating the size per 
transition_matrix_CD4 = transition_2015_2021.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10
  })

transitions_2015_2021 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2015_2021 = pd.DataFrame(transitions_2015_2021, index=[0])
df_transitions_2015_2021_long = pd.melt(df_transitions_2015_2021, var_name='From-To', value_name="Pixel")
df_transitions_2015_2021_long['Pixel'] = df_transitions_2015_2021_long['Pixel'].astype(int)

print(df_transitions_2015_2021_long.head())
df_transitions_2015_2021_long.to_csv('./data/transitions_pixels_2015_2021.csv', index=False, sep=";")


  From-To   Pixel
0     101  189647
1     102   61558
2     103   37005
3     104  103103
4     105   64244


In [44]:
area_changed_CD4_sqm = ee.Image.pixelArea().addBands(transition_2015_2021)
area_changed_CD4_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2015_2021)


# getting the values per class
areas_changed_CD4_sqm_class = area_changed_CD4_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733',
    'tileScale': 16,
    'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD4_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2015_2021.csv', sep=";", index=False)

[{'sum': 4289875.694610237, 'transitions': 101}, {'sum': 1124379.187429488, 'transitions': 102}, {'sum': 580214.1142306833, 'transitions': 103}, {'sum': 1313903.524349497, 'transitions': 104}, {'sum': 1455399.5520504673, 'transitions': 105}, {'sum': 126558.7015292149, 'transitions': 106}, {'sum': 45.491886138916016, 'transitions': 107}, {'sum': 2194661.501520516, 'transitions': 201}, {'sum': 2865198.9699223163, 'transitions': 202}, {'sum': 687141.6807972703, 'transitions': 203}, {'sum': 1335810.903178099, 'transitions': 204}, {'sum': 683394.4421348496, 'transitions': 205}, {'sum': 63611.81289607777, 'transitions': 206}, {'sum': 22.746109008789062, 'transitions': 207}, {'sum': 679603.8573571747, 'transitions': 301}, {'sum': 917730.589187637, 'transitions': 302}, {'sum': 702126.4728506126, 'transitions': 303}, {'sum': 270486.5926770603, 'transitions': 304}, {'sum': 57149.7480805341, 'transitions': 305}, {'sum': 6209.642879486084, 'transitions': 306}, {'sum': 1464872.1124422485, 'transiti

#### Exporting the change mask to Drive and Asset

In [45]:
# exporting to Google drive with GEE API
export_change_mask = ee.batch.Export.image.toDrive(**{
    'image': changed_areas_2015_2017,
    'description': 'change_mask_2015_2017', # TODO: change name here
    'folder': 'masterthesis/change_detection_results', # TODO: change name here
    'scale': 4.77,
    'crs': 'EPSG:32733',
    'region': aoi_geom
})

# starting the process
export_change_mask.start()

# tracking the process
while export_change_mask.active():
  print('Polling for task (id: {}).'.format(export_change_mask.id))
  time.sleep(5)

In [46]:
# # exporting to Google drive with GEE API
# export_change_mask = ee.batch.Export.image.toDrive(**{
#     'image': changed_areas_2017_2019,
#     'description': 'change_mask_2017_2019_test', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))
#   time.sleep(5)

Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id: TIRBP3KVHAMB2HHMI2HF2BHB).
Polling for task (id

In [47]:
# # exporting to Google drive with GEE API
# export_change_mask = ee.batch.Export.image.toDrive(**{
#     'image': changed_areas_2019_2021,
#     'description': 'change_mask_2019_2021', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))
#   time.sleep(5)

In [48]:
# # exporting to Google drive with GEE API
# export_change_mask = ee.batch.Export.image.toDrive(**{
#     'image': changed_areas_2015_2021,
#     'description': 'change_mask_2015_2021', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))
#   time.sleep(5)

In [49]:
# # exporting to Google Asset
# export_change_mask = ee.batch.Export.image.toAsset(**{
#   'image': changed_areas_2015_2017,
#   'description': 'Export Change Mask map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/change_mask_2015_2017', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))

In [50]:
# exporting to Google Asset
export_change_mask = ee.batch.Export.image.toAsset(**{
  'image': changed_areas_2017_2019,
  'description': 'Export Change Mask map',
  'assetId': 'users/s85315/masterthesis/change_detection_results/change_mask_2017_2019_test', # TODO: change name here
  'scale': 4.77,
  'crs': 'EPSG:32733',
  'region': aoi_geom
})

# starting the process
export_change_mask.start()

# tracking the process
while export_change_mask.active():
  print('Polling for task (id: {}).'.format(export_change_mask.id))

Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id: QKQUKOGLKBZYUZHMOYKTNRH4).
Polling for task (id

In [38]:
# # exporting to Google Asset
# export_change_mask = ee.batch.Export.image.toAsset(**{
#   'image': changed_areas_2019_2021,
#   'description': 'Export Change Mask map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/test/change_mask_2019_2021', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_change_mask.start()

# # tracking the process
# while export_change_mask.active():
#   print('Polling for task (id: {}).'.format(export_change_mask.id))

Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id: NHTIQVW7LXQVFNDVR3FONRTC).
Polling for task (id

In [39]:
# exporting to Google Asset
export_change_mask = ee.batch.Export.image.toAsset(**{
  'image': changed_areas_2015_2021,
  'description': 'Export Change Mask map',
  'assetId': 'users/s85315/masterthesis/change_detection_results/change_mask_2015_2021', # TODO: change name here
  'scale': 4.77,
  'crs': 'EPSG:32733',
  'region': aoi_geom
})

# starting the process
export_change_mask.start()

# tracking the process
while export_change_mask.active():
  print('Polling for task (id: {}).'.format(export_change_mask.id))

Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id: 5ZHR6QGZIKR45DWFLXJ6ICOC).
Polling for task (id

In [40]:
# zones = changed_areas_2015_2017.select('classification').eq(1)

# vectors = zones.reduceToVectors(**{
#   'geometry': aoi_geom,
#   'scale': 4.77,
#   'reducer': ee.Reducer.mean()
# })

# display = ee.Image(0).updateMask(0).paint(vectors, '000000', 1)

# visParams2010 = {'min':0, 'max':1, 'palette': ['white', 'red']};


# Map.addLayer(display, {}, 'Vectors')

#### 3.2 Two-Step Accuracy Assessment

#### 3.3 Creating visualization

### 3 CD for TOI 2015 to 2017

#### 3.1 Performing the CD analysis

#### 3.2 Two-Step Accuracy Assessment

### 3 CD for TOI 2015 to 2017

#### 3.1 Performing the CD analysis

#### 3.2 Two-Step Accuracy Assessment

##### Resources for code snippets

https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb \
https://worldbank.github.io/OpenNightLights/tutorials/mod6_6_RF_classifier.html \
https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd \
https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest